<img src="../image/logo-cefet.png" width="300">

<h1> PPCIC Dissertação - 2021
    <br>
    <br>
    DETECÇÃO DE EVENTOS ADVERSOS ATRAVÉS DO TWITTER UTILIZANDO O METAMAP PARA O PORTUGUÊS DO BRASIL </h1>
</br>

<h2> Matéria: CIC1229 - TÓPICOS ESPECIAIS EM ALGORITMOS </h2>
<h2> Aluno: Perciliano </h2>
<h2> Orientadora: Kele Belloze </h2>
<h3> Data: 11/06/2021 </h3>

---

**Autor: Luiz Perciliano** - 
***luiz.perciliano@eic.cefet-rj.br***

- Dados da pesquisa na base SCOPUS dia 13-06-2021
- 1 - String de busca geral
<br>
458 document results
<br>
TITLE-ABS-KEY 
<br>
<b>( ( ( "natural language processing"  AND  ( "adverse drug reaction"  OR  "adverse drug event"  OR  pharmacovigilance  OR  metamap ) ) ) )  </b>
<br>
Essa string 1 contém as 2 e 3.

---------------

- 2 - String de busca inclusa na principal PARA LER E fazer Snowballing principalmente para frente
<br>
6 document results
<br>
( ( "natural language processing"  AND  ( "adverse drug reaction"  OR  "adverse drug event"  OR  pharmacovigilance ) )  AND  metamap )  

---------------

- 3 - String útil
<br>
94 document results
( "natural language processing" AND ( "adverse drug reaction" OR "adverse drug event" OR pharmacovigilance ) AND ( "social media" OR "social network" OR twitter ) )


<br>

**Requisitos**
- RF01 - Analisar bases de dados
- RF02 - Remover duplicatas
- RF03 - Filtrar artigos (retirar os documentos do tipo: conference review, editorial, letter e note)
- RF04 - Identificar artigos para leitura de resumo e conclusão
- RF05 - Identificar artigos para leitura completa

<b>Site do projeto:</b> https:git.com
<br>
**Endereço da dissertação:** https://pt.overleaf.com/project/60731ca2bcfa0afce8ae0cd1

---

# Preparar Infraesrtutura

In [1]:
print(f'Importar as bibliotecas necessárias e mapear a pasta do projeto.')
import os, re
import sys
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk
from unidecode import unidecode
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

Importar as bibliotecas necessárias e mapear a pasta do projeto.


In [2]:
data_inicio = pd.Timestamp.now()
print(data_inicio)

2021-06-16 00:16:17.823455


In [3]:
print(f'Lista do conteúdo da pasta ...')
os.listdir(os.path.join('..','data'))

Lista do conteúdo da pasta ...


['base-qualis-06-06-2021.xlsx',
 'classificacao_qualis_06-06-2021.xls',
 'Evaluation-of-Natural-Language-Processing-NLP-systems-to-annotate-drug-product-labeling-with-MedDRA-terminologyJournal-of-Biomedical-Informatics.pdf',
 'pubmed-10-06-2021.csv',
 'pubmed.csv',
 'PubMed_Timeline_Results_by_Year-10-06-2021.csv',
 'requiremets.txt',
 'resultado_dfs.xlsx',
 'resultado_geral.xlsx',
 'resultado_geral_06-06-2021.xlsx',
 'resultado_pubmed.xlsx',
 'resultado_scopus-luciana-12-06-2021-v2.xlsx',
 'resultado_scopus-luciana-12-06-2021.xlsx',
 'resultado_scopus-luciana.xlsx',
 'resultado_scopus.xlsx',
 'resultado_scopus_10-06-2021.xlsx',
 'scopus-12-06-2021-luciana.csv',
 'scopus-31-05-2021.csv',
 'scopus.csv',
 'stop_word_projeto.csv',
 'stop_word_pt.csv',
 '~$resultado_scopus-luciana-12-06-2021.xlsx']

In [4]:
os.listdir('../Image')

['logo-cefet.png', 'Logo_CVM.png', 'puc-rio-logo.png', 'wordcloud.pdf']

## Verificar, atualizar e instalar se necessário python e módulos 

# Preparar e carregar base de dados

## Conexão e consulta ao SQL Server

## Carregar Planilha - Base Qualis

In [5]:
print('Carregando pesquisa realizada na base Scopus.')
'''
Fonte: https://sucupira.capes.gov.br/sucupira/public/consultas/coleta/veiculoPublicacaoQualis/listaConsultaGeralPeriodicos.jsf
Periódico do quadriênio 2013-2016
'''
arquivo_base_qualis = os.path.join('../data','base-qualis-06-06-2021.xlsx')
raw_data_qualis = pd.read_excel(arquivo_base_qualis)
raw_data_qualis

Carregando pesquisa realizada na base Scopus.


,ISSN,Título,Área de Avaliação,Estrato
0,1981-030X,19&20 (RIO DE JANEIRO),"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS ...",C
1,2236-6695,A BARRIGUDA: REVISTA CIENTÍFICA,"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS ...",B4
2,1413-6090,A ECONOMIA EM REVISTA,"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS ...",B4
3,1516-3210,A&C. REVISTA DE DIREITO ADMINISTRATIVO & CONST...,"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS ...",B4
4,0001-3072,ABACUS (SYDNEY. PRINT),"ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS ...",A2
...,...,...,...,...
131269,1175-5326,ZOOTAXA (AUCKLAND. PRINT),ZOOTECNIA / RECURSOS PESQUEIROS ...,B1
131270,1175-5334,ZOOTAXA (ONLINE),ZOOTECNIA / RECURSOS PESQUEIROS ...,B1
131271,2358-3576,ZOOTECNIA,ZOOTECNIA / RECURSOS PESQUEIROS ...,C
131272,0798-7269,ZOOTECNIA TROPICAL - FONAIAP,ZOOTECNIA / RECURSOS PESQUEIROS ...,B3


## Carregar CSV - Scopus

In [6]:
print('Visualização do dataframe carregado da base scopus.')
arquivo_base_scopus = os.path.join('../data','scopus.csv')
raw_data_scopus = pd.read_csv(arquivo_base_scopus, sep=",", delimiter=None, encoding='utf-8')
raw_data_scopus.head()

Visualização do dataframe carregado da base scopus.


,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Zhang F., Sun B., Diao X., Zhao W., Shu T.",57221913449;57221919849;57202804485;5719859752...,Prediction of adverse drug reactions based on ...,2021,BMC Medical Informatics and Decision Making,21,1,38,NaN,NaN,...,NaN,NaN,33541342.0,English,BMC Med. Informatics Decis. Mak.,Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85100558743
1,"Shen C., Li Z., Chu Y., Zhao Z.",57206995169;57213179826;57191405659;56171564000;,GAR: Graph adversarial representation for adve...,2021,Applied Soft Computing,106,NaN,107324,NaN,NaN,...,NaN,NaN,NaN,English,Appl. Soft Comput.,Article,Final,NaN,Scopus,2-s2.0-85103114472
2,"Shannon G.J., Rayapati N., Corns S.M., Wunsch ...",56275865000;57219795472;6506365939;7006706193;,Comparative study using inverse ontology cogen...,2021,Neural Networks,139,NaN,NaN,86,104,...,NaN,NNETE,33684612.0,English,Neural Netw.,Article,Final,NaN,Scopus,2-s2.0-85102081027
3,"Allam N., Audeh B., Jaulent M.-C., Bousquet C.",57224206480;55452137800;7003346504;8609880700;,Visualising Patterns Associated with Adverse D...,2021,Studies in health technology and informatics,281,NaN,NaN,1110,1111,...,NaN,NaN,34042861.0,English,Stud Health Technol Inform,Article,Final,"All Open Access, Hybrid Gold",Scopus,2-s2.0-85107238693
4,"El-allaly E.-D., Sarrouti M., En-Nahnahi N., O...",57207994980;57170906800;57221862377;55891723400;,MTTLADE: A multi-task transfer learning-based ...,2021,Information Processing and Management,58,3,102473,NaN,NaN,...,NaN,IPMAD,NaN,English,Inf. Process. Manage.,Article,Final,NaN,Scopus,2-s2.0-85100465366


## Carregar CSV - PubMed

In [7]:
print('Carga e Visualização do dataframe')
arquivo_base_pubmed = os.path.join('../data','pubmed.csv')
raw_data_pubmed = pd.read_csv(arquivo_base_pubmed, sep=",", delimiter=None, encoding='utf-8')
raw_data_pubmed.head()

Carga e Visualização do dataframe


,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,30649735,Overview of the First Natural Language Process...,"Jagannatha A, Liu F, Liu W, Yu H.",Drug Saf. 2019 Jan;42(1):99-111. doi: 10.1007/...,Jagannatha A,Drug Saf,2019,2019/01/17,PMC6860017,NIHMS1518894,10.1007/s40264-018-0762-z
1,33245290,Identification of Adverse Drug Event-Related J...,"Ujiie S, Yada S, Wakamiya S, Aramaki E.",JMIR Med Inform. 2020 Nov 27;8(11):e22661. doi...,Ujiie S,JMIR Med Inform,2020,2020/11/27,PMC7732716,NaN,10.2196/22661
2,33215076,Adverse drug event presentation and tracking (...,"Geva A, Stedman JP, Manzi SF, Lin C, Savova GK...",JAMIA Open. 2020 Aug 31;3(3):413-421. doi: 10....,Geva A,JAMIA Open,2020,2020/11/20,PMC7660953,NaN,10.1093/jamiaopen/ooaa031
3,31630063,A systematic review of natural language proces...,"Young IJB, Luz S, Lone N.",Int J Med Inform. 2019 Dec;132:103971. doi: 10...,Young IJB,Int J Med Inform,2019,2019/10/21,NaN,NaN,10.1016/j.ijmedinf.2019.103971
4,33006728,ADE Eval: An Evaluation of Text Processing Sys...,"Bayer S, Clark C, Dang O, Aberdeen J, Brajovic...",Drug Saf. 2021 Jan;44(1):83-94. doi: 10.1007/s...,Bayer S,Drug Saf,2021,2020/10/02,PMC7813736,NaN,10.1007/s40264-020-00996-3


## Analisar dados brutos original

### Analisando base SCOPUS

In [ ]:
#pd.set_option("max_colwidth", 100)

In [8]:
print('Verificando tipos e se tem dados nulos')
raw_data_scopus.info()

Verificando tipos e se tem dados nulos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Authors                        458 non-null    object 
 1   Author(s) ID                   458 non-null    object 
 2   Title                          458 non-null    object 
 3   Year                           458 non-null    int64  
 4   Source title                   458 non-null    object 
 5   Volume                         393 non-null    object 
 6   Issue                          202 non-null    object 
 7   Art. No.                       109 non-null    object 
 8   Page start                     359 non-null    object 
 9   Page end                       348 non-null    object 
 10  Page count                     3 non-null      float64
 11  Cited by                       378 non-null    float64
 12  DOI        

In [9]:
raw_data_scopus.nunique()

Authors                          443
Author(s) ID                     441
Title                            456
Year                              22
Source title                     155
Volume                           172
Issue                             36
Art. No.                         106
Page start                       285
Page end                         295
Page count                         3
Cited by                          71
DOI                              379
Link                             458
Affiliations                     446
Authors with affiliations        453
Abstract                         448
Author Keywords                  318
Index Keywords                   417
Molecular Sequence Numbers         0
Chemicals/CAS                     73
Tradenames                        17
Manufacturers                      1
Funding Details                  209
Funding Text 1                   183
Funding Text 2                    21
Funding Text 3                     1
R

In [10]:
# verificar campos nulos
raw_data_scopus.isnull().sum()

Authors                            0
Author(s) ID                       0
Title                              0
Year                               0
Source title                       0
Volume                            65
Issue                            256
Art. No.                         349
Page start                        99
Page end                         110
Page count                       455
Cited by                          80
DOI                               79
Link                               0
Affiliations                       4
Authors with affiliations          3
Abstract                           0
Author Keywords                  140
Index Keywords                    41
Molecular Sequence Numbers       458
Chemicals/CAS                    362
Tradenames                       439
Manufacturers                    457
Funding Details                  237
Funding Text 1                   275
Funding Text 2                   437
Funding Text 3                   457
R

### Analisando base Qualis

In [11]:
print('Verificando tipos e se tem dados nulos')
raw_data_qualis.info()

Verificando tipos e se tem dados nulos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131274 entries, 0 to 131273
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ISSN               131271 non-null  object
 1   Título             131274 non-null  object
 2   Área de Avaliação  131274 non-null  object
 3   Estrato            131274 non-null  object
dtypes: object(4)
memory usage: 4.0+ MB


In [12]:
raw_data_qualis.nunique()

ISSN                 27618
Título               29838
Área de Avaliação       49
Estrato                  8
dtype: int64

In [13]:
raw_data_qualis.isnull().sum()

ISSN                 3
Título               0
Área de Avaliação    0
Estrato              0
dtype: int64

### Analisando base PUBMED

In [14]:
print('Verificando tipos e se tem dados nulos')
raw_data_pubmed.info()

Verificando tipos e se tem dados nulos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   PMID              252 non-null    int64 
 1   Title             252 non-null    object
 2   Authors           252 non-null    object
 3   Citation          252 non-null    object
 4   First Author      252 non-null    object
 5   Journal/Book      252 non-null    object
 6   Publication Year  252 non-null    int64 
 7   Create Date       252 non-null    object
 8   PMCID             157 non-null    object
 9   NIHMS ID          26 non-null     object
 10  DOI               181 non-null    object
dtypes: int64(2), object(9)
memory usage: 21.8+ KB


In [15]:
raw_data_pubmed.nunique()

PMID                252
Title               251
Authors             248
Citation            252
First Author        215
Journal/Book         61
Publication Year     21
Create Date         204
PMCID               157
NIHMS ID             26
DOI                 181
dtype: int64

In [16]:
# verificar campos nulos
raw_data_pubmed.isnull().sum()

PMID                  0
Title                 0
Authors               0
Citation              0
First Author          0
Journal/Book          0
Publication Year      0
Create Date           0
PMCID                95
NIHMS ID            226
DOI                  71
dtype: int64

## Copiar dataframe para ajustes

In [17]:
## Verificar se os dataframes nao s"ao espelhos
df_scopus = raw_data_scopus.copy()
df_qualis = raw_data_qualis.copy()
df_pubmed = raw_data_pubmed.copy()

print(id(df_scopus),id(df_qualis), id(df_pubmed), id(raw_data_scopus),id(raw_data_qualis), id(raw_data_pubmed))

3034198705160 3034242612232 3034199762696 3034250340104 3034247952904 3034250511368


# Dataframe SCOPUS

## Ajustar de Colunas

### Alterar nome de Colunas

In [18]:
print('Verificar colunas')
df_scopus.columns

Verificar colunas


Index(['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume',
       'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by',
       'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Abstract',
       'Author Keywords', 'Index Keywords', 'Molecular Sequence Numbers',
       'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'Funding Details',
       'Funding Text 1', 'Funding Text 2', 'Funding Text 3', 'References',
       'Correspondence Address', 'Editors', 'Sponsors', 'Publisher',
       'Conference name', 'Conference date', 'Conference location',
       'Conference code', 'ISSN', 'ISBN', 'CODEN', 'PubMed ID',
       'Language of Original Document', 'Abbreviated Source Title',
       'Document Type', 'Publication Stage', 'Open Access', 'Source', 'EID'],
      dtype='object')

In [19]:
columns_scopus = {
    'Authors':'autores_scopus',
    'Author(s) ID':'id_autores',
    'Title':'titulo_artigo',
    'Year':'ano_publicacao_scopus',
    'Source title':'titulo_fonte',
    'Volume':'volume',
    'Issue':'publicado',
    'Art. No.':'numero_artigo',
    'Page start':'inicio_pagina',
    'Page end':'fim_pagina', 
    'Page count':'quantidade_paginas',
    'Cited by':'quantidade_citacoes',
    'DOI':'doi_scopus',
    'Link':'link_scopus',
    'Affiliations':'afiliacoes',
    'Authors with affiliations':'autores_com_filiacoes',
    'Abstract':'resumo',
    'Author Keywords':'palavras_chaves_autor',
    'Index Keywords':'palavras_chave_index',
    'Molecular Sequence Numbers':'numeros_sequencia_molecular' ,
    'Chemicals/CAS':'quimica',
    'Tradenames':'nomes_comerciais',
    'Manufacturers':'fabricantes',
    'Funding Details':'detalhes_financiamento',
    'Funding Text 1':'texto_financiamento_1',
    'Funding Text 2':'texto_financiamento_2',
    'Funding Text 3':'texto_financiamento_3',
    'References':'referencias',
    'Correspondence Address':'endereco_correspondencia',
    'Editors':'editores',
    'Sponsors':'patrocinadores',
    'Publisher':'editor',
    'Conference name':'nome_conferencia',
    'Conference date':'data_conferencia',
    'Conference location':'local_conferencia',
    'Conference code':'codigo_conferencia',
    'ISSN':'issn_scopus',
    'ISBN':'isbn',
    'CODEN':'coden',
    'PubMed ID':'id_pubmed',
    'Language of Original Document':'idioma_original',
    'Abbreviated Source Title':'titulo_abreviado_fonte',
    'Document Type':'tipo_documento',
    'Publication Stage':'etapa_publicacao',
    'Open Access':'acesso_livre',
    'Source':'fonte',
    'EID':'eid',
}

In [20]:
df_scopus = df_scopus.rename(columns=columns_scopus)
df_scopus.head(1)

,autores_scopus,id_autores,titulo_artigo,ano_publicacao_scopus,titulo_fonte,volume,publicado,numero_artigo,inicio_pagina,fim_pagina,...,isbn,coden,id_pubmed,idioma_original,titulo_abreviado_fonte,tipo_documento,etapa_publicacao,acesso_livre,fonte,eid
0,"Zhang F., Sun B., Diao X., Zhao W., Shu T.",57221913449;57221919849;57202804485;5719859752...,Prediction of adverse drug reactions based on ...,2021,BMC Medical Informatics and Decision Making,21,1,38,NaN,NaN,...,NaN,NaN,33541342.0,English,BMC Med. Informatics Decis. Mak.,Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85100558743


### Ajustar / Criar colunas de Páginas

In [21]:
## Criar novas colunas de paginas e + uma para calcular qtde paginas
df_scopus["inicio_pagina_"] = df_scopus["inicio_pagina"]
df_scopus["fim_pagina_"] = df_scopus["fim_pagina"]

In [22]:
## Expressao que pega só dígitos
r = re.compile(r'\D')

In [23]:
df_scopus.inicio_pagina_.replace(r, '', regex = True, inplace=True)
df_scopus.fim_pagina_.replace(r, '', regex = True, inplace=True)

In [27]:
df_scopus.loc[[3,10,2,7], ['inicio_pagina','fim_pagina','quantidade_paginas','inicio_pagina_','fim_pagina_']]

,inicio_pagina,fim_pagina,quantidade_paginas,inicio_pagina_,fim_pagina_
3,1110,1111,NaN,1110,1111
10,585,590,NaN,585,590
2,86,104,NaN,86,104
7,NaN,NaN,NaN,NaN,NaN


In [28]:
print('Ajustar tipagem dos dados')
df_scopus['inicio_pagina_'] = df_scopus['inicio_pagina_'].astype('float64')
df_scopus['fim_pagina_'] = df_scopus['fim_pagina_'].astype('float64')

Ajustar tipagem dos dados


In [29]:
df_scopus['quantidade_paginas_'] = df_scopus['fim_pagina_'] - df_scopus['inicio_pagina_']

In [31]:
df_scopus.loc[[3,10,2,7], ['inicio_pagina','fim_pagina','quantidade_paginas','inicio_pagina_','fim_pagina_','quantidade_paginas_']]

,inicio_pagina,fim_pagina,quantidade_paginas,inicio_pagina_,fim_pagina_,quantidade_paginas_
3,1110,1111,NaN,1110.0,1111.0,1.0
10,585,590,NaN,585.0,590.0,5.0
2,86,104,NaN,86.0,104.0,18.0
7,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_scopus.sort_values('quantidade_paginas_',ascending=False, )[['issn_scopus','quantidade_paginas_']]

,issn_scopus,quantidade_paginas_
42,22132198,9349.0
205,22132198,6696.0
329,NaN,263.0
223,03029743,131.0
111,18650929,26.0
...,...,...
429,1942597X,NaN
430,1942597X,NaN
433,15594076,NaN
438,14712105,NaN


### Criar nova coluna com data de conferencia

## Colunas para lower

In [ ]:
## Passar colunas a serem trabalhadas para lower case


In [ ]:

## dar strip frente e atras, 
## Carregar novas strings ingles para eliminar das palavras
## novas palavras ingles para append
## Coluna Issue para categóriaca (0 ou 1) para aplicar ML

In [ ]:
## Coluna Issue para categóriaca (0 ou 1)
#dataset_df.num_andares.replace({'-': 1}, inplace=True)

## Ajustar tipagem de dados

### Convertendo as colunas de datas para o formato datetime

### Convertendo as colunas para categóricas

In [ ]:
print('Ajustar tipagem dos dados')
df_scopus['fabricantes'] = df_scopus['fabricantes'].astype('category')
df_scopus['acesso_livre'] = df_scopus['acesso_livre'].astype('category')

### Inclusão de Colunas

In [ ]:
df_scopus['palavras_chaves_titulo']=None # variavel para receber as palavras + importantes em um artigo
df_scopus['palavras_chaves_resumo']=None # variavel para receber as palavras + importantes em um artigo

#### Carregar palavras_chaves_titulo

In [ ]:
df_scopus.loc[
    (df_scopus['titulo_artigo'].str.contains('twitter'))|
    (df_scopus['titulo_artigo'].str.contains('metamap'))|
    (df_scopus['titulo_artigo'].str.contains('NLP'))|
    (df_scopus['titulo_artigo'].str.contains('natural language processing')),'palavras_chaves_titulo'] = "twitter|metamap|nlp"

In [ ]:
df_scopus.query('palavras_chaves_titulo=="twitter|metamap|nlp"')['titulo_artigo'].count()

#### Carregar palavras_chaves_resumo

## NAO - Filtrar dataframe

In [ ]:
df_scopus.head()

In [ ]:
## Criar Pipeline com Airflow

## Analisar dataframe tratado

In [ ]:
print('Verificando tipos e se tem dados nulos')
df_scopus.info()

In [ ]:
### analisar estes números máximos e mínimos ?????????
print('Resumo Estatístico de Campos Numéricos')
df_scopus.describe()

In [ ]:
# ver qtd no excel =NÚM.CARACT(O2)
print('Coluna com maior qtde de caracteres')
df_scopus['titulo_artigo'].apply(str).map(len).max()

In [ ]:
# analisar alguns campos

In [ ]:
## tirar os espaços das colunas strings a serem trabalhadas

## Visualizações

### Analisando tipos de acessos dos arquivos

In [ ]:
## pegar os 10 + patrocinadores
df_scopus.acesso_livre.unique()

In [ ]:
# Groupby by
acesso_livre = df_scopus.groupby("acesso_livre")

# Summary statistic of all
acesso_livre.describe().head()

In [ ]:
# pegar as 10 +
plt.figure(figsize=(15,10))
acesso_livre.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Tipos de acesso aos artigos")
plt.ylabel("Quantidade de Artigos")
plt.show()

### Analisando tipos de documentos

In [ ]:
# Groupby by
tipo_documento = df_scopus.groupby("tipo_documento")

# Summary statistic of all
tipo_documento.describe().head()

In [ ]:
# pegar as 10 +
plt.figure(figsize=(15,10))
tipo_documento.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Locais de Conferências")
plt.ylabel("Quantidade de Locais de Conferência")
plt.show()

### Analisando Locais de Conferencia

In [ ]:
# Groupby by
local_conferencia = df_scopus.groupby("local_conferencia")

# Summary statistic of all
local_conferencia.describe().head()

In [ ]:
# pegar as 10 +
plt.figure(figsize=(15,10))
local_conferencia.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Locais de Conferências")
plt.ylabel("Quantidade de Locais de Conferência")
plt.show()

### Analisando Editores

In [ ]:
# Groupby by
editor = df_scopus.groupby("editor")

# Summary statistic of all
editor.describe().head()

In [ ]:
plt.figure(figsize=(15,10))
editor.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Fabricantes")
plt.ylabel("Número de Artigos")
plt.show()

### Analisando artigos por Ano

In [ ]:
# Groupby by
ano = df_scopus.groupby("ano_publicacao_scopus")

# Summary statistic of all
ano.describe().head()

In [ ]:
plt.figure(figsize=(15,10))
ano.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Anos")
plt.ylabel("Número de Artigos")
plt.show()

### Analisando os idiomas

In [ ]:
# Groupby by
idioma = df_scopus.groupby("idioma_original")

# Summary statistic of all
idioma.describe().head()

In [ ]:
plt.figure(figsize=(15,10))
idioma.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Idioma Original")
plt.ylabel("Número de Artigos")
plt.show()

In [ ]:
print('Visão geral em gráfico')
sns.pairplot(df_scopus)

In [ ]:
## qtde de artigos financiados e nao financiados - criar coluna

In [ ]:
## qtde de artigos patrociandores e nao patrociandores - criar coluna

In [ ]:
## criar regex para identificar emails da coluna editores

In [ ]:
## qtde de artigos por editores - unicos

In [ ]:
## qtde de artigos por local de conferencia - unicos

In [ ]:
## qtde de artigos por idioma_original - unicos

In [ ]:
## qtde de artigos por tipo de documentos - unicos

## Nuvem de Palavras

 ### Nuvem de Palavras dos Títulos

In [ ]:
import nltk
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

print(stopwords)

# apend outras palavras

In [ ]:
print('Carregar novo dataframe de palavras apenas com dados da coluna texto, ou seja, uma série.')
#df_scopus['issn_scopus'] = df_scopus.issn_scopus.str.upper()
palavras_titulo = df_scopus['titulo_artigo'].str.lower()
palavras_titulo

In [ ]:
print(f'O objeto palavras é do tipo {type(palavras_titulo)} e tem o shape de {palavras_titulo.shape}')

In [ ]:
print(f'Criando Nuvem de Palavras sem tratamento para ter ideia do corpus.')
# Variável recebe conteúdo do dataframe palavras concatenando cada conteúdo do texto separando por espaço
wordcloud_palavras_titulo = " ".join(s for s in palavras_titulo)
#wordcloud_palavras

In [ ]:
print(f'Quantidade de palavras no corpus: {len(wordcloud_palavras_titulo)} e seu tipo atual é{type(wordcloud_palavras_titulo)}')

In [ ]:
print(f'Criação da nuvem de palavras sem tratamento nos textos')
wordcloud_titulo = WordCloud(stopwords=stopwords,
                      background_color='black', width=1600,                            
                      height=800).generate(wordcloud_palavras_titulo)

In [ ]:
#Gráfico Nuvem de Palavras
# Gerando o grafico 
# Variáveis do gráfico 
path_image = '../image/'

titulo = 'Nuvem de Palavras'
#eixo_x = ''
#eixo_y = ''
image = path_image+'wordcloud'
extensao_arquivo = '.pdf'

#Gráfico
fig, ax = plt.subplots(figsize=(16,8))            
ax.imshow(wordcloud_titulo, interpolation='bilinear')
ax.set_axis_off()         

#legendas
#ax.legend(title='Legenda', loc=4, fontsize=9)
ax.set_title(titulo, fontsize=18)
#ax.set_xlabel(eixo_x, fontsize=9)
#ax.set_ylabel(eixo_y, fontsize=9)

#salvar imagens
plt.savefig(image+extensao_arquivo) #, format='pdf', dpi=300, transparent=True)
plt.tight_layout() 
plt.show()

### Nuvem de Palavras dos Resumos

In [ ]:
print('Carregar novo dataframe de palavras apenas com dados da coluna texto, ou seja, uma série.')
palavras_resumo = df_scopus['resumo'].str.lower()
palavras_resumo

In [ ]:
print(f'Criando Nuvem de Palavras sem tratamento para ter ideia do corpus.')
# Variável recebe conteúdo do dataframe palavras concatenando cada conteúdo do texto separando por espaço
wordcloud_palavras_resumo = " ".join(s for s in palavras_resumo)
#wordcloud_palavras

In [ ]:
print(f'Quantidade de palavras no corpus: {len(wordcloud_palavras_resumo)} e seu tipo atual é{type(wordcloud_palavras_resumo)}')

In [ ]:
print(f'Criação da nuvem de palavras sem tratamento nos textos')
wordcloud_resumo = WordCloud(stopwords=stopwords,
                      background_color='black', width=1600,                            
                      height=800).generate(wordcloud_palavras_resumo)

In [ ]:
#Gráfico Nuvem de Palavras
path_image = '../image/'

titulo = 'Nuvem de Palavras'
#eixo_x = ''
#eixo_y = ''
image = path_image+'wordcloud'
extensao_arquivo = '.pdf'

#Gráfico
fig, ax = plt.subplots(figsize=(16,8))            
ax.imshow(wordcloud_resumo, interpolation='bilinear')
ax.set_axis_off()         

#legendas
#ax.legend(title='Legenda', loc=4, fontsize=9)
ax.set_title(titulo, fontsize=18)
#ax.set_xlabel(eixo_x, fontsize=9)
#ax.set_ylabel(eixo_y, fontsize=9)

#salvar imagens
plt.savefig(image+extensao_arquivo) #, format='pdf', dpi=300, transparent=True)
plt.tight_layout() 
plt.show()

### Nuvem de Palavras do Conteúdo dos textos

# Dataframe Qualis

## Ajustar de Colunas

### Alterar nome de colunas

In [ ]:
print('Verificar colunas')
df_qualis.columns

In [ ]:
columns_qualis = {
    'ISSN': 'issn_qualis',
    'Título':'titulo_periodico',
    'Área de Avaliação':'area_avaliacao', 
    'Estrato':'estrato',
}

In [ ]:
df_qualis = df_qualis.rename(columns=columns_qualis)
df_qualis.head(1)

## Ajustar tipagem de dados

### Converter colunas para categórica

In [ ]:
print('Ajustar tipagem dos dados')
df_qualis['estrato'] = df_qualis['estrato'].astype('category')

### Filtrar base Qualis

In [ ]:
print(df_qualis['area_avaliacao'].unique())

### Remover as areas abaixo

In [ ]:

#%%timeit
df_remove = df_qualis.loc[
    (df_qualis['area_avaliacao'] == 'ADMINISTRAÇÃO PÚBLICA E DE EMPRESAS, CIÊNCIAS CONTÁBEIS E TURISMO') |
    (df_qualis['area_avaliacao'] == 'ARTES') |
    (df_qualis['area_avaliacao'] == 'ANTROPOLOGIA / ARQUEOLOGIA                        ') |
    (df_qualis['area_avaliacao'] == 'ARQUITETURA, URBANISMO E DESIGN') |
    (df_qualis['area_avaliacao'] == 'ASTRONOMIA / FÍSICA                               ') |
    (df_qualis['area_avaliacao'] == 'BIODIVERSIDADE') |
    (df_qualis['area_avaliacao'] == 'CIÊNCIAS AGRÁRIAS I                               ') |
    (df_qualis['area_avaliacao'] == 'CIÊNCIA DE ALIMENTOS                              ') |
    (df_qualis['area_avaliacao'] == 'CIÊNCIA POLÍTICA E RELAÇÕES INTERNACIONAIS        ') |
    (df_qualis['area_avaliacao'] == 'CIÊNCIAS AGRÁRIAS I                               ') |
    (df_qualis['area_avaliacao'] == 'CIÊNCIAS AMBIENTAIS') |   
    (df_qualis['area_avaliacao'] == 'CIÊNCIAS DA RELIGIÃO E TEOLOGIA') |
    (df_qualis['area_avaliacao'] == 'COMUNICAÇÃO E INFORMAÇÃO') |
    (df_qualis['area_avaliacao'] == 'DIREITO                                           ') |
    (df_qualis['area_avaliacao'] == 'ECONOMIA                                          ') |
    (df_qualis['area_avaliacao'] == 'EDUCAÇÃO                                          ') |
    (df_qualis['area_avaliacao'] == 'ENSINO') |
    (df_qualis['area_avaliacao'] == 'EDUCAÇÃO FÍSICA                                   ') |
    (df_qualis['area_avaliacao'] == 'ENGENHARIAS I                                     ') |
    (df_qualis['area_avaliacao'] == 'ENGENHARIAS II                                    ') |
    (df_qualis['area_avaliacao'] == 'ENGENHARIAS III                                   ') |
    (df_qualis['area_avaliacao'] == 'ENGENHARIAS IV                                    ' ) |
    (df_qualis['area_avaliacao'] == 'GEOCIÊNCIAS                                       ' ) |
    (df_qualis['area_avaliacao'] == 'FILOSOFIA') |
    (df_qualis['area_avaliacao'] == 'GEOGRAFIA                                         ') |
    (df_qualis['area_avaliacao'] == 'HISTÓRIA                                          ') |
    (df_qualis['area_avaliacao'] == 'INTERDISCIPLINAR                                  ') |
    (df_qualis['area_avaliacao'] == 'LINGUíSTICA E LITERATURA') |
    (df_qualis['area_avaliacao'] == 'MATERIAIS                                         ') |
    (df_qualis['area_avaliacao'] ==  'MEDICINA VETERINÁRIA                              ') |
    (df_qualis['area_avaliacao'] ==  'NUTRIÇÃO') |
    (df_qualis['area_avaliacao'] == 'PLANEJAMENTO URBANO E REGIONAL / DEMOGRAFIA       ') |
    (df_qualis['area_avaliacao'] == 'SERVIÇO SOCIAL                                    ') |
    (df_qualis['area_avaliacao'] == 'SOCIOLOGIA                                        ') |
    (df_qualis['area_avaliacao'] ==  'ZOOTECNIA / RECURSOS PESQUEIROS                   ') 
 
]

df_qualis_filtrado = df_qualis.drop(df_remove.index)
df_qualis_filtrado

## Analisar dataframe tratado

In [ ]:
print('Verificando tipos e se tem dados nulos')
df_qualis_filtrado.info()

In [ ]:
print('Resumo Estatístico de Campos Numéricos')
df_qualis_filtrado.describe()

In [ ]:
# ver qtd no excel =NÚM.CARACT(O2)
print('Coluna com maior qtde de caracteres')
df_qualis_filtrado['titulo_periodico'].apply(str).map(len).max()

## Visualizações Qualis

### Anáilise Area de Avaliação

In [ ]:
df_qualis_filtrado.columns


In [ ]:
# Groupby by
area_avaliacao_all = df_qualis_filtrado.groupby("area_avaliacao")

# Summary statistic of all
area_avaliacao_all.describe().head()

In [ ]:
# pegar as 10 +
plt.figure(figsize=(15,10))
area_avaliacao_all.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Área de Avaliação geral - Base Qualis")
plt.ylabel("Quantidade de Artigos")
plt.grid()
plt.show()

In [ ]:
### Gerando gráfico Tipo de Extensões

#variáveis do gráfico
titulo = 'Quantidade de Periodicos'
eixo_x = 'Tipos de Periodicos'
eixo_y = 'Quantidade de Periodicos'
data_hora = datetime.datetime.now()
path = os.path.join('..','image')
img_extensao = '.png'
image = ''

# Matplotlib
fig, ax = plt.subplots(figsize=(15,5))
plt.bar(df_qualis_filtrado['area_avaliacao'].value_counts().index, df_qualis_filtrado['area_avaliacao'].value_counts())

#Legendas e eixos
#ax.legend(title='Legenda', loc=4, fontsize=20)
ax.set_title(titulo, fontsize=20)
ax.set_xlabel(eixo_x, fontsize=15)
ax.set_ylabel(eixo_y, fontsize=15)
plt.yticks(rotation=45) ## agrupar datas por meses, trimestre ou ano, etc

plt.grid()

#Salvar imagem
#plt.savefig(path+image, dpi=300)
plt.tight_layout()
plt.show()

plt.show()

# Dataframe PubMEd

## Ajustar de Colunas

### Alterar nome de Colunas

In [ ]:
## criar uma coluna fonte com valor = pubmed

In [ ]:
print('Verificar colunas')
df_pubmed.columns

In [ ]:
columns_scopus = {
    'PMID':'id_pubmed',
    'Title':'titulo_artigo',
    'Authors':'autores_pubmed',
    'Citation':'citar',
    'First Author':'primeiro_autor',
    'Journal/Book':'jornal_book',
    'Publication Year':'ano_publicacao_pubmed',
    'Create Date':'data_criacao_pubmed', 
    'PMCID':'id_pmc', 
    'NIHMS ID':'id_nihms', 
    'DOI':'doi_pubmed',
}

In [ ]:
df_pubmed = df_pubmed.rename(columns=columns_scopus)
df_pubmed.head(2)

## Ajustar tipagem de dados

In [ ]:
df_pubmed.info()

### Convertendo as colunas de datas para o formato datetime

In [ ]:
#Convertendo as colunas de datas para o formato datetime
colunas_datas = df_pubmed.columns[df_pubmed.columns.str.contains('DT_|dt\|data|DATA|Data', regex=True)]

In [ ]:
#convertendo cada coluna de colunas_datas para o formato datetime
for coluna in colunas_datas:
    df_pubmed[coluna] = pd.to_datetime(df_pubmed[coluna], format='%Y-%m-%d')

In [ ]:
## nao funfou, verificar
df_pubmed.info()

### Convertendo as colunas para categóricas

In [ ]:
print('Ajustar tipagem dos dados')
df_pubmed['jornal_book'] = df_pubmed['jornal_book'].astype('category')

In [ ]:
## nao funfou, verificar
df_pubmed.info()

# Dataframe CAPES

# Concatenar dataframes de bases indexadas

In [ ]:
#https://www.acervolima.com.br/2020/08/concat-e-append-do-pandas-como-usar.html
## cancatenar ambas as bases em um df
## este novo df tem q ter no total o somatório de ambos
## remover as duplicatas com id_pubmed e titulo aritigo


In [ ]:
qtde_scopus = df_scopus.titulo_artigo.count()
qtde_pubmed = df_pubmed.titulo_artigo.count()
print(f'A quantidade de artigos na base scopus é: {qtde_scopus}')
print(f'A quantidade de artigos na base pubmed é: {qtde_pubmed}')
print(f'O total de artigos concatedos tem que ser {qtde_pubmed+qtde_scopus}')

In [ ]:
frames = [df_scopus, df_pubmed]

In [ ]:
resultado = pd.concat(frames)

In [ ]:
type(resultado)

In [ ]:
resultado

In [ ]:
resultado.columns

In [ ]:
#resultado = resultado.drop_duplicates(['titulo_artigo', 'id_pubmed'], keep='last')

## testar unitariamente com cada coluna
resultado = resultado.drop_duplicates(['titulo_artigo', 'id_pubmed']) = 581 
resultado = resultado.drop_duplicates(['id_pubmed'])
print(resultado)

# Unir / Merge de bases com Qualis

In [ ]:
#df_qualis["issn_qualis_ajustado"] = 
df_qualis_filtrado["issn_qualis"].replace('-','',regex=True, inplace=True)

In [ ]:
df_qualis_filtrado["issn_qualis"]

In [ ]:
df_scopus['issn_scopus'] = df_scopus.issn_scopus.str.upper()
df_qualis_filtrado['issn_qualis'] = df_qualis_filtrado.issn_qualis.str.upper()

In [ ]:
df_scopus['issn_scopus']

In [ ]:
df_qualis_filtrado['issn_qualis']

In [ ]:
## criar um dataframe q une o links_regulamentos_fundos + allLines, ligados pelo nome do arquivo
df_bases = df_scopus.merge(df_qualis_filtrado, left_on='issn_scopus', right_on='issn_qualis', how='left', indicator=True)
df_bases

In [ ]:
#df_scopus.merge(df_qualis, left_on='issn_scopus', right_on='issn_qualis', how='left', validate='m:m')

In [ ]:
df_bases[['autores_scopus','titulo_artigo','area_avaliacao','issn_scopus','issn_qualis','estrato']]

In [ ]:
print(type(df_bases['_merge'].unique()))

In [ ]:
print(df_bases['_merge'].unique())

In [ ]:
df_bases.nunique()

In [ ]:
df_bases.info()

In [ ]:
# Criar colunas
df_bases['leitura_Resumo']=None 
df_bases['leitura_Conclusao']=None 
df_bases['leitura_completa']=None 

## Visualizações Base Geral

### Analisando Nomes Comerciais

In [ ]:
#Colocar + para baixo após criar este df

# Groupby by
nomes_comerciais = df_bases.groupby("nomes_comerciais")

# Summary statistic of all
nomes_comerciais.describe().head()

In [ ]:
plt.figure(figsize=(15,10))
nomes_comerciais.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Nomes Comerciais")
plt.ylabel("Número de Artigos")
plt.show()

In [ ]:
# Groupby by
area_avaliacao = df_bases.groupby("area_avaliacao")

# Summary statistic of all
area_avaliacao.describe().head()

In [ ]:
# pegar as 10 +
plt.figure(figsize=(15,10))
area_avaliacao.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Área de Avaliação")
plt.ylabel("Quantidade de Artigos")
plt.show()

### Análise de Titulo Periodico

In [ ]:
# Groupby by
titulo_periodico = df_bases.groupby("titulo_periodico")

# Summary statistic of all
titulo_periodico.describe().head()

In [ ]:
# pegar as 10 +
plt.figure(figsize=(15,10))
titulo_periodico.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Título dos Periódicos")
plt.ylabel("Quantidade de Periodicos")
plt.show()

# Exportação do resultado para Excel

## Planilha Scopus

In [ ]:
df_scopus.columns

In [ ]:
## Gerar planilha com colunas especificas
#resultado_geral = os.path.join('../data','scopus-31-05-2021.csv')
resultado_scopus = os.path.join('../data','resultado_scopus.xlsx')

df_xlsx_scopus = pd.DataFrame(df_scopus, columns = [
       'issn_scopus','id_pubmed','titulo_artigo', 'ano_publicacao_scopus','quantidade_citacoes',
       'tipo_documento','nome_conferencia','editores', 'patrocinadores', 'editor',
       'autores_scopus','id_autores', 'titulo_fonte','volume', 'publicado', 'numero_artigo',
       'inicio_pagina', 'fim_pagina','quantidade_paginas','inicio_pagina_','fim_pagina_','quantidade_paginas_',
       'link_scopus','afiliacoes', 'autores_com_filiacoes', 'resumo',
       'palavras_chaves_autor', 'palavras_chave_index',
       'numeros_sequencia_molecular', 'quimica', 'nomes_comerciais',
       'fabricantes',
       'detalhes_financiamento', 'texto_financiamento_1','texto_financiamento_2', 'texto_financiamento_3',
       'referencias', 'endereco_correspondencia', 
       'data_conferencia', 'local_conferencia',
       'codigo_conferencia', 'isbn', 'coden',
       'idioma_original', 'titulo_abreviado_fonte', 
       'etapa_publicacao', 'acesso_livre', 'fonte', 'eid','doi_scopus' 
    ,])
df_xlsx_scopus = df_xlsx_scopus.to_excel(resultado_scopus,index=False, encoding='utf-8', header=True)

## Planilha PubMed

In [ ]:
df_pubmed.columns

In [ ]:
## Gerar planilha com colunas especificas
#resultado_geral = os.path.join('../data','scopus-31-05-2021.csv')
resultado_pubmed = os.path.join('../data','resultado_pubmed.xlsx')

df_xlsx_pubmed = pd.DataFrame(df_pubmed, columns = [
    'id_pubmed', 'titulo_artigo','ano_publicacao_pubmed','autores_pubmed', 'citar_autor',
    'primeiro_autor', 'jornal_book','data_criacao_pubmed',
    'id_pmc', 'id_nihms', 'doi_pubmed'
    ,])
df_xlsx_pubmed = df_xlsx_pubmed.to_excel(resultado_pubmed,index=False, encoding='utf-8', header=True)

In [ ]:
resultado.columns

In [ ]:
## Planilha de base indexada

In [ ]:
## Gerar planilha com colunas especificas
#resultado_geral = os.path.join('../data','scopus-31-05-2021.csv')
resultado_dfs = os.path.join('../data','resultado_dfs.xlsx')

df_xlsx_dfs = pd.DataFrame(resultado, columns = [
'autores_scopus', 'id_autores', 'titulo_artigo',
       'ano_publicacao_scopus', 'titulo_fonte', 'volume', 'publicado',
       'numero_artigo', 'inicio_pagina', 'fim_pagina', 'quantidade_paginas',
       'quantidade_citacoes', 'doi_scopus', 'link_scopus', 'afiliacoes',
       'autores_com_filiacoes', 'resumo', 'palavras_chaves_autor',
       'palavras_chave_index', 'numeros_sequencia_molecular', 'quimica',
       'nomes_comerciais', 'fabricantes', 'detalhes_financiamento',
       'texto_financiamento_1', 'texto_financiamento_2',
       'texto_financiamento_3', 'referencias', 'endereco_correspondencia',
       'editores', 'patrocinadores', 'editor', 'nome_conferencia',
       'data_conferencia', 'local_conferencia', 'codigo_conferencia',
       'issn_scopus', 'isbn', 'coden', 'id_pubmed', 'idioma_original',
       'titulo_abreviado_fonte', 'tipo_documento', 'etapa_publicacao',
       'acesso_livre', 'fonte', 'eid', 'inicio_pagina_', 'fim_pagina_',
       'quantidade_paginas_', 'autores_pubmed', 'citar', 'primeiro_autor',
       'jornal_book', 'ano_publicacao_pubmed', 'data_criacao_pubmed', 'id_pmc',
       'id_nihms', 'doi_pubmed'
    ,])
df_xlsx_dfs = df_xlsx_dfs.to_excel(resultado_dfs,index=False, encoding='utf-8', header=True)

In [ ]:
df_bases.columns

In [ ]:
## Gerar planilha com colunas especificas
#resultado_geral = os.path.join('../data','scopus-31-05-2021.csv')
resultado_geral = os.path.join('../data','resultado_geral.xlsx')

df_xlsx = pd.DataFrame(df_bases, columns = 
        ['titulo_artigo', 'ano_publicacao','titulo_periodico', 'area_avaliacao', 'estrato','id_autores', 'titulo_fonte',
       'volume', 'publicado', 'numero_artigo', 'inicio_pagina', 'fim_pagina',
       'quantidade_paginas', 'quantidade_citacoes', 'doi', 'link_scopus',
       'autores','afiliacoes', 'autores_com_filiacoes', 'resumo',
       'palavras_chaves_autor', 'palavras_chave_index',
       'numeros_sequencia_molecular', 'quimica', 'nomes_comerciais',
       'fabricantes', 'detalhes_financiamento', 'texto_financiamento_1',
       'texto_financiamento_2', 'texto_financiamento_3', 'referencias',
       'endereco_correspondencia', 'editores', 'patrocinadores', 'editor',
       'nome_conferencia', 'data_conferencia', 'local_conferencia',
       'codigo_conferencia', 'issn_scopus', 'isbn', 'coden', 'id_pubmed',
       'idioma_original', 'titulo_abreviado_fonte', 'tipo_documento',
       'etapa_publicacao', 'acesso_livre', 'fonte', 'eid', 'issn_qualis','_merge'
       'leitura_Resumo','leitura_Conclusao','leitura_completa'
         ,])
df_xlsx = df_xlsx.to_excel(resultado_geral,index=False, encoding='utf-8', header=True)
#df_xlsx_1 = df_xlsx_1.to_excel(r'C:/Users/luizp/jupyter-notebook/SisCRI-ML/data/REQ-002-publico-alvo-Geral_v6.xlsx',index=True, encoding='utf-8', header=True)

# Analises

In [ ]:
data_fim = pd.Timestamp.now()
print(data_inicio)
print(data_fim)